In [1]:
#Build a Decision Tree Classifier to predict Employee Attrition using features like Age, Department, Job_Satisfaction, Work_Life_Balance,
#and Years_at_Company. Evaluate using Accuracy, Precision, Recall, and ROC-AUC.

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,f1_score, make_scorer
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [13]:
df = pd.read_csv("synthetic_employee_data.csv") 
X = df[['Age', 'Department', 'Job_Satisfaction', 'Work_Life_Balance', 'Years_at_Company']]
y = df['Attrition']

# Convert categorical columns (if any)
X = pd.get_dummies(X)

In [21]:
y = y.map({'No': 0, 'Yes': 1})

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [27]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))

Accuracy: 1.0
Precision: 1.0
Recall: 1.0
ROC-AUC: 1.0


In [29]:
#Compare Logistic Regression, Random Forest, and XGBoost classifiers for predicting Attrition. 
#Use 10-fold cross-validation and report the average F1-score for each. Which model generalizes better?

In [37]:
models = {
    "Logistic": LogisticRegression(max_iter=500),
    "RandomForest": RandomForestClassifier()
}

for name, clf in models.items():
    scores = cross_val_score(clf, X, y, cv=10, scoring=make_scorer(f1_score))
    print(f"{name} F1-score: {scores.mean():.4f}")

Logistic F1-score: 0.5867
RandomForest F1-score: 0.9667


In [39]:
#Use SMOTE to handle class imbalance (if exists after train-test split) and retrain a Random Forest Classifier.
#Compare model performance before and after applying SMOTE.

In [43]:
smote = SMOTE()
X_res, y_res = smote.fit_resample(X_train, y_train)

rf = RandomForestClassifier()
rf.fit(X_res, y_res)
y_pred_res = rf.predict(X_test)

print("After SMOTE - Accuracy:", accuracy_score(y_test, y_pred_res))

After SMOTE - Accuracy: 0.9975


In [47]:
#Perform Hyperparameter tuning on a Random Forest Classifier using GridSearchCV. Tune parameters like n_estimators, max_depth, and 
#min_samples_split.Report the best parameter set and corresponding ROC-AUC.

In [51]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}

In [53]:
grid = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='roc_auc')
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_split': [2, 5],
                         'n_estimators': [50, 100]},
             scoring='roc_auc')

In [55]:
print("Best params:", grid.best_params_)
print("Best ROC-AUC:", grid.best_score_)

Best params: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 50}
Best ROC-AUC: 1.0


In [57]:
#Use Chi-Square test for feature selection and identify the top 5 significant features contributing to employee attrition.
#Retrain a classifier using only these features and report performance metrics.

In [63]:
X_numeric = X.copy()
X_numeric = X_numeric[X_numeric >= 0]

In [65]:
selector = SelectKBest(score_func=chi2, k=5)
X_new = selector.fit_transform(X_numeric, y)

selected_features = X.columns[selector.get_support()]
print("Top 5 Features:", selected_features.tolist())

Top 5 Features: ['Job_Satisfaction', 'Work_Life_Balance', 'Years_at_Company', 'Department_Admin', 'Department_Sales']


In [67]:
X_selected = X[selected_features]
X_train_sel, X_test_sel, y_train_sel, y_test_sel = train_test_split(X_selected, y, test_size=0.2, random_state=42)
model_sel = RandomForestClassifier()
model_sel.fit(X_train_sel, y_train_sel)
print("Selected Features Accuracy:", accuracy_score(y_test_sel, model_sel.predict(X_test_sel)))

Selected Features Accuracy: 1.0
